In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/ML Projects/Brain Cancer Detection/Dataset'

In [ ]:
%cd /content/gdrive/MyDrive/ML Projects/Brain Cancer Detection/Dataset

/content/gdrive/MyDrive/ML Projects/Brain Cancer Detection/Dataset


In [ ]:
!ls

kaggle.json


In [ ]:
!kaggle datasets download -d jakeshbohaju/brain-tumor

 65% 9.00M/14.0M [00:00<00:00, 14.1MB/s]
100% 14.0M/14.0M [00:00<00:00, 22.0MB/s]


In [ ]:
!ls

brain-tumor.zip  kaggle.json


In [ ]:
!mkdir Input-Data
!mv brain-tumor.zip Input-Data

In [ ]:
%cd Input-Data/
!ls

/content/gdrive/My Drive/ML Projects/Brain Cancer Detection/Dataset/Input-Data
brain-tumor.zip


In [ ]:
!unzip brain-tumor.zip

Archive:  brain-tumor.zip
  inflating: Brain Tumor.csv         
  inflating: Brain Tumor/Brain Tumor/Image1.jpg  
  inflating: Brain Tumor/Brain Tumor/Image10.jpg  
  inflating: Brain Tumor/Brain Tumor/Image100.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1000.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1001.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1002.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1003.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1004.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1005.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1006.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1007.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1008.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1009.jpg  
  inflating: Brain Tumor/Brain Tumor/Image101.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1010.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1011.jpg  
  inflating: Brain Tumor/Brain Tumor/Image1012.jpg  
  inflating: Brain Tumor/Brain Tumor/Image

In [2]:
# Imports
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class BrainTumourDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        
        image = Image.open(img_path + ".jpg")
        image = image.resize((120,120))
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [5]:
# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 20

In [6]:
# Load Data
dataset = BrainTumourDataset(
    csv_file="/content/gdrive/MyDrive/ML Projects/Brain Cancer Detection/Dataset/Input-Data/Brain Tumor.csv",
    root_dir="/content/gdrive/MyDrive/ML Projects/Brain Cancer Detection/Dataset/Input-Data/Brain Tumor/Brain Tumor",
    transform=transforms.ToTensor(),
)

In [7]:
dataset.__len__()

3762

In [ ]:
im_age,label = dataset.__getitem__(15)
print(label)
plt.imshow(im_age)
print(im_age.shape)
print(im_age)

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset, [3262, 500])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [ ]:
# Our CNN based neural architecture 
# Let's build a simple one with only Convolutional, Linear 
# and dropout layers
class CNN_Model(nn.Module):
    # Here we define the neural architecture 
    def __init__(self):
        
        super(CNN_Model, self).__init__() # Initialize the nn module 
        self.conv1 = nn.Conv2d(3, 6, 5) # 116
        self.conv2 = nn.Conv2d(6, 16, 9) # 108
        self.pool = nn.MaxPool2d(2, 2) # 54
        self.conv3 = nn.Conv2d(16, 17, 10) # 45
        self.conv4 = nn.Conv2d(17, 18, 10) # 36
        self.conv5 = nn.Conv2d(18, 19, 9) # 10
        self.fc1 = nn.Linear(19 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 50)
        self.fc3 = nn.Linear(50, 20)
        self.fc4 = nn.Linear(20, 2)
        
        # Convolutional Layers
        # What shape/dimensions the first layer is going to see? 
        # Do we need to have some padding for a kernel_size = 3?  
        # Input Features = 1 x 28 x 28
        # Output Features = ???
        # Shape of a Convolutional Layer = (W - K + 2P)
        #                                  ------------ + 1
        #                                       S
        # where, 
        #       W = Width/Height of previous layer = 28
        #       K = Filter Size = 3
        #       P = Padding = 0
        #       S = Stride = 1(default)
        # Therefore, 
        #           if padding = 0
        #           Output Shape = ((28 - 3 + 2*0)/1)+1 = 26 
        # We want the dimensions to stay the same so that there is no 
        # loss of information when performing the convolution. 
        # Hence, 
        #       if padding = 1
        #       Output Shape = ((28 - 3 + 2*1)/1)+1 = 28
        # ---- FILL IN ----

        # Linear Layers 
        # What shape the first linear layer is going see?
        # What are the total number of features given out by conv2?
        # Features = 16 x 28 x 28 = 12544
        # Therefore,           
        # ---- FILL IN ----

        # Last linear layer should output 10 features as we are 
        # Classifying the images in 10 categories 
        # ---- FILL IN ----

        # Dropout 
        # ---- FILL IN ----

    # Here we define the 'forward behaviour' of our neural architecture 
    def forward(self, x):
        # -> n, 3, 120, 120
        x = F.relu(self.conv1(x))            # -> n, 6, 116, 116 
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 54, 54
        x = F.relu(self.conv3(x))           # -> n, 17, 45, 45
        x = self.pool(F.relu(self.conv4(x)))  # -> n, 18, 18, 18
        x = self.pool(F.relu(self.conv5(x)))  # -> n, 19, 5, 5

        x = x.view(-1, 19 * 5 * 5)            # -> n, 475
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 50
        x = F.relu(self.fc3(x))               # -> n, 20
        x = self.fc4(x)                       # -> n, 2
        return x

        # This is also the place where we add ACTIVATION functions 
        # ---- FILL IN ----

        # Remember that when passing image_batch through the Linear layers, 
        # PyTorch expects: 
        # >>> torch.Size([12, 256]) -> example values 
            # 2d: [batch_size, num_features (aka: C * H * W)]
            # use for nn.Linear() input.   
        # Therefore, we need to 'flatten' image_batch
        # image_batch = image_batch.view(batch_size, -1) --> batch size ???
        # ---- FILL IN ----

        # Let's add the dropout too 
        # ---- FILL IN ----

        # Final Layer of the network 
        # ---- FILL IN ----

        # The output from the final layer is a tensor with 10 'logits'

In [ ]:
model = CNN_Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 0.7042623138895222
Cost at epoch 1 is 0.7023764948050181
Cost at epoch 2 is 0.700725288367739
Cost at epoch 3 is 0.6992620434246811
Cost at epoch 4 is 0.6979639267220217
Cost at epoch 5 is 0.6968332172608843
Cost at epoch 6 is 0.6958306326585657
Cost at epoch 7 is 0.6949435069280512
Cost at epoch 8 is 0.6941504829070148
Cost at epoch 9 is 0.6934510387626349
Cost at epoch 10 is 0.6928385250708636
Cost at epoch 11 is 0.6922940454062294
Cost at epoch 12 is 0.6918135957390654
Cost at epoch 13 is 0.6913915346650517
Cost at epoch 14 is 0.6910040302603853
Cost at epoch 15 is 0.6906739233755598
Cost at epoch 16 is 0.6903731466508379
Cost at epoch 17 is 0.6901170845125236
Cost at epoch 18 is 0.6898758575028064
Cost at epoch 19 is 0.6896763151767207


In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [ ]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 1792 / 3262 with accuracy 54.94
Checking accuracy on Test Set
Got 287 / 500 with accuracy 57.40
